# Specializations

Now, we have a basic FEM solver with rudimentary features. We can now start to add more features to the solver, in mini-projects that are done in smaller groups in parallel.

Each project should contain some new feature implementation, a demo notebook and corresponding unit tests. The feature should be implemented in a way that it could be used in the final (merged) project by all other users.

The following list is a colletion of ideas and partially pretty roughly sketched only:

### Autodiff for Derivatives

We have used numerical differentiation as a fall back to compute the derivatives of shape functions. Alternatively we can directly compute the derivatives alongside the shapes itself using an autodiff data type. 

To this end we need to introduce a datatype which re-implements standard arithmetic operations. It holds a value and a derivative value. The arithmetic operations then compute value and derivative using standard rules of differentiation, e.g. product rule:
```python
x = AD(ip[0],(1,0)) # the expression corresponding to x has value ip[0] and derivative (1,0)
y = AD(ip[1],(0,1)) # the expression corresponding to x has value ip[0] and derivative (1,0)
z = x*y # the expression corresponding to z
# z.val == x.val * y.val
# z.der == x.val * y.der + x.der * y.val
```

The task of this mini-project is to implement such a datatype and use it to compute the derivatives of shape functions.


### Boundary integrals

So far, all considered integrals have been integrals on $\Omega$. However, other use cases are integrals on $\partial \Omega$. For example, the boundary integral that arises from non-homogeneous Neumann boundary conditions.

The task of this mini-project is to implement a boundary integral: 
* This requires to change the assembly-functions to distinguish between loops over boundary and volume elements and to apply corresponding  `FormIntegrator`s only the corresponding parts
* This again requires to change the `FormIntegrator` interface to allow for different integrals on boundary and volume elements.
* Further a transformation from 1D to 2D, respectively 0D to 1D is required to map the boundary integral to the reference element (of co-dimension 1).



### Non-homogeneous Dirichlet boundary conditions

So far, we have only implemented homogenenous Dirichlet boundary conditions (and potentially only with a hack). In this task we want to implement non-homogeneous Dirichlet boundary conditions more systematically in several steps:

* Projection of a given function on a given set of boundary elements, using the Oswald interpolation operator which applys a simple $L^2$ projection on all relevant boundary elements. For dofs that appear several times the corresponding mean value is used. Hence, here, the $L^2$ projection needs to be implemented.

* Given the projection, a particular solution for a homogenization strategy can be computed. This is then used to setup a linear system for a correction PDE solution that satisfies homogeneous Dirichlet boundary conditions.

For this project further details will be given. 

### Oswald interpolation

Implementation of an Oswald projection operator:
 * Apply a simple $L^2$ projection on all relevant boundary elements. 
 * For dofs that appear several times the corresponding mean value is used. 
 
Hence, here, the $L^2$ projection needs to be implemented.

### Nodal interpolation operator

Implementation of a nodal interpolation operator for Lagrange-type FESpaces.

### `MeshFunction` arithmetics

Given several `MeshFunction`s (e.g. exact solution and discrete solution, i.e. a `GlobalFunction` and a `FEFunction`) we want to be able to compute the sum, difference, product, ... of these `MeshFunction`s as a new `MeshFunction`. This requires to implement the corresponding arithmetics for `MeshFunction`s.

### Consider a different scalar PDE: Convection-Diffusion equations

Consider the (reaction-)convection-diffusion equation
$$
-\nabla \cdot (\alpha \nabla u) + \beta \cdot \nabla u + \gamma u = f
$$
in $\Omega$, equipped with suitable boundary conditions.

Implement a FEM solver for this problem and carry out numerical experiments to investigate the influence of the convection term on the solution.

### Consider different linear solvers

So far we only used a direct solver to solve the PDE (which is not that bad in 1D and 2D, but not optimal in 3D). In this task we want to consider different linear solvers, e.g. a CG solver with preconditioning.

### Adaptive mesh refinement

Implement an adaptive mesh refinement strategy for the Poisson problem. This requires to implement a posteriori error estimation and a mesh refinement strategy. 